<a href="https://colab.research.google.com/github/chloe-jeon/2020-Kundaje-Lab/blob/master/Exon_Intron_UTR_First_Gene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
genes = files.upload() #protein_coding.gtf

Saving protein_coding.gtf to protein_coding.gtf


KeyboardInterrupt: ignored

In [ ]:
!cat protein_coding.gtf | head -n 19 > first_gene.gtf
!cat first_gene.gtf

chr1	HAVANA	gene	65419	71585	.	+	.	gene_id "ENSG00000186092.6"; gene_type "protein_coding"; gene_name "OR4F5"; level 2; hgnc_id "HGNC:14825"; havana_gene "OTTHUMG00000001094.1";
chr1	HAVANA	transcript	65419	71585	.	+	.	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; level 2; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.1"; havana_transcript "OTTHUMT00000003223.1";
chr1	HAVANA	exon	65419	65433	.	+	.	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; exon_number 1; exon_id "ENSE00003812156.1"; level 2; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.1"; havana_transcript "OTT

In [ ]:
file = open('first_gene.gtf')
first_gene = file.read()
print(first_gene)

chr1	HAVANA	gene	65419	71585	.	+	.	gene_id "ENSG00000186092.6"; gene_type "protein_coding"; gene_name "OR4F5"; level 2; hgnc_id "HGNC:14825"; havana_gene "OTTHUMG00000001094.1";
chr1	HAVANA	transcript	65419	71585	.	+	.	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; level 2; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.1"; havana_transcript "OTTHUMT00000003223.1";
chr1	HAVANA	exon	65419	65433	.	+	.	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; exon_number 1; exon_id "ENSE00003812156.1"; level 2; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.1"; havana_transcript "OTT

In [ ]:
!awk -v OFS="\t" '{print $1, $3, $4, $5}' first_gene.gtf | awk -v OFS="\t" '{if ($2 == "transcript" || $2 == "exon" || $2 == "CDS") print $1, $2, $3, $4}' > first_gene_transcripts.bed
f = open('first_gene_transcripts.bed')
first_gene_transcripts = f.readlines()
print(first_gene_transcripts)

#I'm assuming that any part of an exon that isn't CDS is start/end UTR

['chr1\ttranscript\t65419\t71585\n', 'chr1\texon\t65419\t65433\n', 'chr1\texon\t65520\t65573\n', 'chr1\tCDS\t65565\t65573\n', 'chr1\texon\t69037\t71585\n', 'chr1\tCDS\t69037\t70005\n', 'chr1\ttranscript\t69055\t70108\n', 'chr1\texon\t69055\t70108\n', 'chr1\tCDS\t69091\t70005\n']


In [ ]:
!cat first_gene_transcripts.bed

chr1	transcript	65419	71585
chr1	exon	65419	65433
chr1	exon	65520	65573
chr1	CDS	65565	65573
chr1	exon	69037	71585
chr1	CDS	69037	70005
chr1	transcript	69055	70108
chr1	exon	69055	70108
chr1	CDS	69091	70005


In [ ]:
#make a 4D nested list
#[[65419, 7185, [65419, 65433], [65520, 65573, [65565, 65573]], [69037, 71585, [69037, 70005]]], [69055, 70108, [69055, 70108, [69091, 70005]]]]
#[transcript1   exon1---------  exon2----------==============-  exon3----------==============-|  transcript2    exon1----------==============-|]

transcripts_list = []
tlist_index = -1
elist_index = -1
for x in first_gene_transcripts:
  temp = x.split()
  if (temp[1] == 'transcript'):
    tlist_index += 1
    elist_index = -1
    transcripts_list.append([int(temp[2]), int(temp[3])])
  elif (temp[1] == 'exon'):
    elist_index += 1
    transcripts_list[tlist_index].append([int(temp[2]), int(temp[3])])
  elif (temp[1] == 'CDS'):
    transcripts_list[tlist_index][elist_index+2].append([int(temp[2]), int(temp[3])])
transcripts_list

[[65419,
  71585,
  [65419, 65433],
  [65520, 65573, [65565, 65573]],
  [69037, 71585, [69037, 70005]]],
 [69055, 70108, [69055, 70108, [69091, 70005]]]]

In [ ]:
#transcripts_list = [[65419, 71585, [65419, 65433], [65520, 65573, [65565, 65573]], [69037, 71585, [69037, 70005]]], [69055, 70108, [69055, 70108, [69091, 70005]]]]

In [ ]:
endpoints = []
for x in first_gene_transcripts:
  endpoints.append(int(x.split()[2]))
  endpoints.append(int(x.split()[3]))
endpoints

[65419,
 71585,
 65419,
 65433,
 65520,
 65573,
 65565,
 65573,
 69037,
 71585,
 69037,
 70005,
 69055,
 70108,
 69055,
 70108,
 69091,
 70005]

In [ ]:
endpoints = list(dict.fromkeys(endpoints))
endpoints.sort()
endpoints

[65419, 65433, 65520, 65565, 65573, 69037, 69055, 69091, 70005, 70108, 71585]

In [ ]:
#[65419, 65433, 65520, 65565, 65573, 69037, 69055, 69091, 70005, 70108, 71585]

In [ ]:
#make a list with segment start point, segment end point, 
#number CDS, number start UTR, number end UTR, number intron, 
#percent CDS, percent start UTR, percent end UTR, percent intron
#note: number columns are 0, percent columns should be blank for now (placeholder 0)
#note: will turn this into a bed file later

segments_list = []

for i in range(int(len(endpoints) -1)):
  segments_list.append([endpoints[i], endpoints[i + 1], 0, 0, 0, 0, 0, 0, 0, 0])
segments_list

[[65419, 65433, 0, 0, 0, 0, 0, 0, 0, 0],
 [65433, 65520, 0, 0, 0, 0, 0, 0, 0, 0],
 [65520, 65565, 0, 0, 0, 0, 0, 0, 0, 0],
 [65565, 65573, 0, 0, 0, 0, 0, 0, 0, 0],
 [65573, 69037, 0, 0, 0, 0, 0, 0, 0, 0],
 [69037, 69055, 0, 0, 0, 0, 0, 0, 0, 0],
 [69055, 69091, 0, 0, 0, 0, 0, 0, 0, 0],
 [69091, 70005, 0, 0, 0, 0, 0, 0, 0, 0],
 [70005, 70108, 0, 0, 0, 0, 0, 0, 0, 0],
 [70108, 71585, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
#for each segment (a.k.a. item in list from previous cell):
###pick a point in the segment
###for each transcript:
#####for each exon in the transcript:
#######if it is between the endpoints of the exon:
#########exon+=1 (it is an exon for this transcript)
#######else
#########if it is between the endpoints of the transcript:
###########intron+=1 (it is an intron for this transcript)

#note: this no longer fully reflects the below code

In [ ]:
for x in segments_list:
  point = x[0] + 1
  #print(point)
  for y in transcripts_list:
    exon = False
    for j in range(2, len(y)):
      if (y[j][0] < point) and (point < y[j][1]): #exon
        if (len(y[j]) > 2 and y[j][2][0] < point) and (point < y[j][2][1]): #CDS
          x[2] += 1
        elif (len(y[j]) > 2 and y[j][2][0] > point): #start UTR, since the segment is part of the exon but before the CDS
          x[3] += 1
        else: #end UTR
          x[4] += 1
        exon = True
        #print("point " + str(point) + " is in an exon when j = " + str(j))
    if not exon and (y[0] < point) and (point < y[1]): #intron
      x[5] += 1
      #print("point " + str(point) + " is in an intron.")
segments_list

[[65419, 65433, 0, 0, 1, 0, 0, 0, 0, 0],
 [65433, 65520, 0, 0, 0, 1, 0, 0, 0, 0],
 [65520, 65565, 0, 1, 0, 0, 0, 0, 0, 0],
 [65565, 65573, 1, 0, 0, 0, 0, 0, 0, 0],
 [65573, 69037, 0, 0, 0, 1, 0, 0, 0, 0],
 [69037, 69055, 1, 0, 0, 0, 0, 0, 0, 0],
 [69055, 69091, 1, 1, 0, 0, 0, 0, 0, 0],
 [69091, 70005, 2, 0, 0, 0, 0, 0, 0, 0],
 [70005, 70108, 0, 0, 2, 0, 0, 0, 0, 0],
 [70108, 71585, 0, 0, 1, 0, 0, 0, 0, 0]]

In [ ]:
for x in segments_list:
  x[6] = x[2] / (x[2] + x[3] + x[4] + x[5])
  x[7] = x[3] / (x[2] + x[3] + x[4] + x[5])
  x[8] = x[4] / (x[2] + x[3] + x[4] + x[5])
  x[9] = x[5] / (x[2] + x[3] + x[4] + x[5])
segments_list

[[65419, 65433, 0, 0, 1, 0, 0.0, 0.0, 1.0, 0.0],
 [65433, 65520, 0, 0, 0, 1, 0.0, 0.0, 0.0, 1.0],
 [65520, 65565, 0, 1, 0, 0, 0.0, 1.0, 0.0, 0.0],
 [65565, 65573, 1, 0, 0, 0, 1.0, 0.0, 0.0, 0.0],
 [65573, 69037, 0, 0, 0, 1, 0.0, 0.0, 0.0, 1.0],
 [69037, 69055, 1, 0, 0, 0, 1.0, 0.0, 0.0, 0.0],
 [69055, 69091, 1, 1, 0, 0, 0.5, 0.5, 0.0, 0.0],
 [69091, 70005, 2, 0, 0, 0, 1.0, 0.0, 0.0, 0.0],
 [70005, 70108, 0, 0, 2, 0, 0.0, 0.0, 1.0, 0.0],
 [70108, 71585, 0, 0, 1, 0, 0.0, 0.0, 1.0, 0.0]]

In [ ]:
segments_list = [[65419, 65433, 0, 0, 1, 0, 0.0, 0.0, 1.0, 0.0], [65433, 65520, 0, 0, 0, 1, 0.0, 0.0, 0.0, 1.0], [65520, 65565, 0, 1, 0, 0, 0.0, 1.0, 0.0, 0.0], [65565, 65573, 1, 0, 0, 0, 1.0, 0.0, 0.0, 0.0], [65573, 69037, 0, 0, 0, 1, 0.0, 0.0, 0.0, 1.0], [69037, 69055, 1, 0, 0, 0, 1.0, 0.0, 0.0, 0.0], [69055, 69091, 1, 1, 0, 0, 0.5, 0.5, 0.0, 0.0], [69091, 70005, 2, 0, 0, 0, 1.0, 0.0, 0.0, 0.0], [70005, 70108, 0, 0, 2, 0, 0.0, 0.0, 1.0, 0.0], [70108, 71585, 0, 0, 1, 0, 0.0, 0.0, 1.0, 0.0]]
segments_list

[[65419, 65433, 0, 0, 1, 0, 0.0, 0.0, 1.0, 0.0],
 [65433, 65520, 0, 0, 0, 1, 0.0, 0.0, 0.0, 1.0],
 [65520, 65565, 0, 1, 0, 0, 0.0, 1.0, 0.0, 0.0],
 [65565, 65573, 1, 0, 0, 0, 1.0, 0.0, 0.0, 0.0],
 [65573, 69037, 0, 0, 0, 1, 0.0, 0.0, 0.0, 1.0],
 [69037, 69055, 1, 0, 0, 0, 1.0, 0.0, 0.0, 0.0],
 [69055, 69091, 1, 1, 0, 0, 0.5, 0.5, 0.0, 0.0],
 [69091, 70005, 2, 0, 0, 0, 1.0, 0.0, 0.0, 0.0],
 [70005, 70108, 0, 0, 2, 0, 0.0, 0.0, 1.0, 0.0],
 [70108, 71585, 0, 0, 1, 0, 0.0, 0.0, 1.0, 0.0]]

In [ ]:
from google.colab import files
files.upload() #pcg_windows_sorted_200.bed and pcg_windows_sorted_rem-not200.bed

Saving pcg_windows_sorted_rem-not200.bed to pcg_windows_sorted_rem-not200.bed


KeyboardInterrupt: ignored

In [ ]:
!cat pcg_windows_sorted_200.bed | head -n 62

chr1	65418	65618
chr1	65518	65718
chr1	65618	65818
chr1	65718	65918
chr1	65818	66018
chr1	65918	66118
chr1	66018	66218
chr1	66118	66318
chr1	66218	66418
chr1	66318	66518
chr1	66418	66618
chr1	66518	66718
chr1	66618	66818
chr1	66718	66918
chr1	66818	67018
chr1	66918	67118
chr1	67018	67218
chr1	67118	67318
chr1	67218	67418
chr1	67318	67518
chr1	67418	67618
chr1	67518	67718
chr1	67618	67818
chr1	67718	67918
chr1	67818	68018
chr1	67918	68118
chr1	68018	68218
chr1	68118	68318
chr1	68218	68418
chr1	68318	68518
chr1	68418	68618
chr1	68518	68718
chr1	68618	68818
chr1	68718	68918
chr1	68818	69018
chr1	68918	69118
chr1	69018	69218
chr1	69118	69318
chr1	69218	69418
chr1	69318	69518
chr1	69418	69618
chr1	69518	69718
chr1	69618	69818
chr1	69718	69918
chr1	69818	70018
chr1	69918	70118
chr1	70018	70218
chr1	70118	70318
chr1	70218	70418
chr1	70318	70518
chr1	70418	70618
chr1	70518	70718
chr1	70618	70818
chr1	70718	70918
chr1	70818	71018
chr1	70918	71118
chr1	71018	71218
chr1	71118	71318
chr1	71218	714

In [ ]:
!cat pcg_windows_sorted_rem-not200.bed | head -n 60

 chr1	65418	65618
chr1	65518	65718
chr1	65618	65818
chr1	65718	65918
chr1	65818	66018
chr1	65918	66118
chr1	66018	66218
chr1	66118	66318
chr1	66218	66418
chr1	66318	66518
chr1	66418	66618
chr1	66518	66718
chr1	66618	66818
chr1	66718	66918
chr1	66818	67018
chr1	66918	67118
chr1	67018	67218
chr1	67118	67318
chr1	67218	67418
chr1	67318	67518
chr1	67418	67618
chr1	67518	67718
chr1	67618	67818
chr1	67718	67918
chr1	67818	68018
chr1	67918	68118
chr1	68018	68218
chr1	68118	68318
chr1	68218	68418
chr1	68318	68518
chr1	68418	68618
chr1	68518	68718
chr1	68618	68818
chr1	68718	68918
chr1	68818	69018
chr1	68918	69118
chr1	69018	69218
chr1	69118	69318
chr1	69218	69418
chr1	69318	69518
chr1	69418	69618
chr1	69518	69718
chr1	69618	69818
chr1	69718	69918
chr1	69818	70018
chr1	69918	70118
chr1	70018	70218
chr1	70118	70318
chr1	70218	70418
chr1	70318	70518
chr1	70418	70618
chr1	70518	70718
chr1	70618	70818
chr1	70718	70918
chr1	70818	71018
chr1	70918	71118
chr1	71018	71218
chr1	71118	71318
chr1	71218	71

In [ ]:
#all of the above was in case I need the files to use in Python
#now I realize I actually want to convert segments_list into a bed file, which will allow me to use awk
#maybe it would've been faster to use awk to do the whole for loop thing? then I wouldn't have to convert to bed
#but how do I do that?

In [ ]:
#I forgot to subtract 1 from the start points in segments_list (1-indexed vs 0-indexed).
#That'll be an issue here.

In [ ]:
#option 1 - use Python for loops

#for window in windows_list #Python 2D list version of one of the above bed files
###for segment in segments_list:
#####if segment endpoint is greater than: #too early
#######continue
#####if segment is too late: #too late
#######break
#####

In [ ]:
#option 2 - use awk

#awk windows bed file
###